In [100]:
import pandas as pd
import json
import os
from tqdm import tqdm

In [103]:
with open('/opt/ml/input/dialogue_data/Training/Train/KAKAO_1000_01.json','r') as f:
    temp = json.load(f)

In [104]:
temp.keys()

dict_keys(['dataset', 'licenses', 'info'])

In [105]:
temp['dataset']

{'identifier': 71459,
 'name': 'KAKAO_1000_01_set',
 'src_path': '/data/file/cubeManager/PROJECT001/53/txt20211007113435043254/KAKAO_1000_01_set/',
 'label_path': '/data/file/cubeManager/PROJECT001/53/txt20211007113435043254/KAKAO_1000_01_set/',
 'category': 2,
 'type': 0}

In [106]:
temp['licenses']

{'name': 'Apache License 1.0',
 'url': 'http://www.apache.org/licenses/LICENSE-1.0'}

In [136]:
temp['info'][0]['annotations']['lines'][0]['speechAct']

'(단언) 주장하기'

In [156]:
def dialogue_sort(example):
    ex_dict = {}
    speech_Act = []
    speech_Cat = []
    speech_CatDeT = []
    norm_texts = []
    morph_list = []
    # shortened variables
    infos = example['info'][0]
    annots = infos['annotations']
    # common properties
    ex_dict['id'] = infos['id']
    ex_dict['media'] = infos['medianame']
    ex_dict['date'] = infos['date']
    ex_dict['category'] = infos['category']
    # annotation properties
    ex_dict['subject'] = annots['subject']
    ex_dict['speaker_type'] = annots['speaker_type']
    ex_dict['raw_text'] = annots['text']
    ex_dict['raw_lines'] = annots['lines']
    # line properties
    for i in range(len(annots['lines'])):
        # 화행 정보 분류
        speech_Act.append(annots['lines'][i]['speechAct'])
        # 0 : 미분류, 2: 지시, 1: 단언, 3: 언약, 4: 표현, 5: (선언/위임하기), 6: 턴토크 사인(관습적 반응)
        if annots['lines'][i]['speechAct'][:4] == '(지시)':
            speech_Cat.append(2)
            if annots['lines'][i]['speechAct'] == '(지시) 명령/요구하기':
                speech_CatDeT.append(21)
            elif annots['lines'][i]['speechAct'] == '(지시) 부탁하기':
                speech_CatDeT.append(22)
            elif annots['lines'][i]['speechAct'] == '(지시) 질문하기':
                speech_CatDeT.append(23)
            elif annots['lines'][i]['speechAct'] == '(지시) 충고/제안하기':
                speech_CatDeT.append(24)
            else:
                speech_CatDeT.append(20)
        elif annots['lines'][i]['speechAct'][:4] == '(단언)':
            speech_Cat.append(1)
            if annots['lines'][i]['speechAct'] == '(단언) 반박하기':
                speech_CatDeT.append(11)
            elif annots['lines'][i]['speechAct'] == '(단언) 주장하기':
                speech_CatDeT.append(12)
            elif annots['lines'][i]['speechAct'] == '(단언) 진술하기':
                speech_CatDeT.append(13)
            else:
                speech_CatDeT.append(10)
        elif annots['lines'][i]['speechAct'][:4] == '(언약)':
            speech_Cat.append(3)
            if annots['lines'][i]['speechAct'] == '(언약) 거절하기':
                speech_CatDeT.append(31)
            elif annots['lines'][i]['speechAct'] == '(언약) 약속하기(제3자와)/(개인적 수준)':
                speech_CatDeT.append(32)
            else:
                speech_CatDeT.append(30)
        elif annots['lines'][i]['speechAct'][:4] == '(표현)':
            speech_Cat.append(4)
            if annots['lines'][i]['speechAct'] == '(표현) 감정하기':
                speech_CatDeT.append(41)
            elif annots['lines'][i]['speechAct'] == '(표현) 긍정감정 표현하기':
                speech_CatDeT.append(42)
            elif annots['lines'][i]['speechAct'] == '(표현) 부정감정 표현하기':
                speech_CatDeT.append(43)
            elif annots['lines'][i]['speechAct'] == '(표현) 사과하기':
                speech_CatDeT.append(44)
            elif annots['lines'][i]['speechAct'] == '(표현) 인사하기':
                speech_CatDeT.append(45)
            else:
                speech_CatDeT.append(40)
        elif annots['lines'][i]['speechAct'] == '(선언/위임하기)':
            speech_Cat.append(5)
            speech_CatDeT.append(5)
        elif annots['lines'][i]['speechAct'] == '턴토크 사인(관습적 반응)':
            speech_Cat.append(6)
            speech_CatDeT.append(6)
        else:
            speech_Cat.append(0)
            speech_CatDeT.append(0)
        norm_texts.append(annots['lines'][i]['norm_text'])
        morph_list.append(annots['lines'][i]['morpheme'])
    # line properties to dict
    ex_dict['norm_texts'] = norm_texts
    ex_dict['text_morphs'] = morph_list
    ex_dict['speech_Act'] = speech_Act
    ex_dict['speech_Cat'] = speech_Cat
    ex_dict['speech_CatDet'] = speech_CatDeT
    return ex_dict

In [157]:
train_path = "/opt/ml/input/dialogue_data/Training/Train"
valid_path = "/opt/ml/input/dialogue_data/Validation/Valid"

In [158]:
dialogue_sort(temp)

{'id': 71543,
 'media': '카카오톡',
 'date': '2021-10-14',
 'category': '일상대화',
 'subject': '여행',
 'speaker_type': '1:1',
 'raw_text': '1 : 다음 주 날씨 너무 당황스러워\n2 : 왜 비 계속 온대?\n1 : 아니, 최저기온이 8도까지 떨어진대\n2 : 아니 가을 어디갔어?\n1 : 이러다가 패딩 꺼내야되는 거 아니야?\n2 : 진짜 경량패딩입어야 될지도 모르겠다\n1 : 난 오늘 반바지 입었다고\n2 : 하하 반바지 입다가 경량패딩으로 바로 가\n1 : 어느 온도에 장단을 맞춰야할지\n2 : 아침엔 겨울 낮엔 가을인 듯해\n1 : 감기 조심해야겠다\n2 : 응 일교차가 심하니까\n1 : 그래 감기조심하고 이따가 또 이야기하자~!',
 'raw_lines': [{'id': 1,
   'text': '1 : 다음 주 날씨 너무 당황스러워',
   'norm_text': '다음 주 날씨 너무 당황스러워',
   'speaker': {'id': '1번', 'sex': '여성', 'age': '30대'},
   'speechAct': '(단언) 주장하기',
   'morpheme': '다음/NNG+주/NNG+날씨/NNG+너무/MAG+당황/NNG+스러워/XSA+EC'},
  {'id': 2,
   'text': '2 : 왜 비 계속 온대?',
   'norm_text': '왜 비 계속 온대?',
   'speaker': {'id': '2번', 'sex': '여성', 'age': '30대'},
   'speechAct': '(지시) 질문하기',
   'morpheme': '왜/MAG+비/XPN+계속/NNG+온대/VV+EF'},
  {'id': 3,
   'text': '1 : 아니, 최저기온이 8도까지 떨어진대',
   'norm_text': '아니, 최저기온이 8도까지 떨어진대',
   'speaker': {'id': '1번', 'sex': '여성', 'age': '3

In [166]:
data_list = []
extra_list = []
# train path / valid path switch
for (root, directories, files) in os.walk(valid_path):
    for file in tqdm(files):
        if '.json' in file:
            file_path = os.path.join(root, file)
            with open(file_path,'r') as f:
                try:
                    temp = json.load(f)
                    data_list.append(dialogue_sort(temp))
                except:
                    extra_list.append(file_path)

100%|██████████| 10962/10962 [00:02<00:00, 4697.69it/s]


In [160]:
len(data_list)

87689

In [161]:
len(extra_list)

1

In [162]:
pd.DataFrame(extra_list)[0][0]

'/opt/ml/input/dialogue_data/Training/Train/KAKAO_1648_13.json'

In [164]:
pd.DataFrame(data_list).head(5)

,id,media,date,category,subject,speaker_type,raw_text,raw_lines,norm_texts,text_morphs,speech_Act,speech_Cat,speech_CatDet
0,132160,카카오톡,2021-11-03,일상대화,교통,다자간 대화,1 : 버스타고 기분 좋았던 경험?\n2 : 나는 버스타고 정처없이 떠돌 때 기분 ...,"[{'id': 1, 'text': '1 : 버스타고 기분 좋았던 경험?', 'nor...","[버스타고 기분 좋았던 경험?, 나는 버스타고 정처없이 떠돌 때 기분 좋더라, 사람...",[버스/NNG+타/VV+고/EC+기분/NNG+좋/VA+았/EP+던/ETM+경험/NN...,"[(지시) 질문하기, (표현) 긍정감정 표현하기, (표현) 긍정감정 표현하기, (단...","[2, 4, 4, 1, 1, 4, 2, 1, 2, 4, 2, 4]","[23, 42, 42, 13, 12, 42, 23, 11, 23, 42, 23, 43]"
1,46838,카카오톡,2021-10-06,일상대화,미용,1:1,1 : 요즘 모공관리가 고민이야.\n1 : 모공관리하는 너만의 비법이 있니?\n2 ...,"[{'id': 1, 'text': '1 : 요즘 모공관리가 고민이야.', 'norm...","[요즘 모공관리가 고민이야., 모공관리하는 너만의 비법이 있니?, 나는 가끔 스크럽...","[요즘/NNG+모공/NNG+관리/NNG+가/JKS+고민/NNG+이/VCP+야/EF,...","[(단언) 주장하기, (지시) 질문하기, (단언) 진술하기, (단언) 진술하기, (...","[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[12, 23, 13, 13, 12, 13, 13, 13, 13, 13, 12, 1..."
2,133919,카카오톡,2021-11-03,일상대화,미용,다자간 대화,1 : 피부에 젤 중요한 건 선크림이다!\n2 : 맞아 키키 선크림 매일 바르고 다...,"[{'id': 1, 'text': '1 : 피부에 젤 중요한 건 선크림이다!', '...","[피부에 젤 중요한 건 선크림이다!, 맞아 키키 선크림 매일 바르고 다니지, 선크림...",[피부/NNG+에/JKB+젤/MAG+중요/NNG+한/XSA+ETM+건/NNB+JX+...,"[(단언) 진술하기, (단언) 진술하기, (단언) 진술하기, (단언) 주장하기, (...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, ...","[13, 13, 13, 12, 13, 12, 13, 13, 23, 12, 12, 2..."
3,90019,카카오톡,2021-10-25,일상대화,교통,1:1,1 : 회사랑은 위치 가깝나 ?\n2 : 차 타고 30분 정도?\n1 : 아 꽤 거...,"[{'id': 1, 'text': '1 : 회사랑은 위치 가깝나 ?', 'norm_...","[회사랑은 위치 가깝나 ?, 차 타고 30분 정도?, 아 꽤 거리가 있는 편이네 ?...","[회사/NNG+랑/JKB+은/JX+위치/NNG+가깝/VA+나/EF, 차/NNG+타/...","[(지시) 질문하기, (단언) 주장하기, (단언) 진술하기, (단언) 주장하기, (...","[2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, ...","[23, 12, 13, 12, 12, 12, 12, 23, 12, 12, 12, 1..."
4,81331,카카오톡,2021-10-20,일상대화,미용,1:1,1 : 요즘 미장원에 손님들이 많아요?\n2 : 코로나 초창기에는 얼마 없었는데 요...,"[{'id': 1, 'text': '1 : 요즘 미장원에 손님들이 많아요?', 'n...","[요즘 미장원에 손님들이 많아요?, 코로나 초창기에는 얼마 없었는데 요즘엔 많아졌어...",[요즘/NNG+미장원/NNG+에/JKB+손님/NNG+들/XSN+이/JKS+많/VA+...,"[(지시) 질문하기, (단언) 진술하기, (단언) 주장하기, (단언) 주장하기, (...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[23, 13, 12, 12, 12, 12, 12, 13, 13, 12]"


In [167]:
pd.DataFrame(data_list).to_csv('/opt/ml/input/dialogue_data/valid.csv')